In [2]:
import requests
import json
import random
import time
import pickle

model1 = "dRITphi3"

template = {
  "Answer": "",
  "Score Lexical Semantics": "",
  "Score Predicate-Argument Structure": "",
  "Score Logic": "",
  "Score Knowledge & Common Sense": ""
}


# DEV de MultiNLI

In [3]:
import pandas as pd

In [4]:
df_t=pd.read_pickle("../predictions/RL_MultiNLI_DEVM_600.pickle")

In [5]:
df_t

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
0,2,0.333333,0.000000,0.333333,0.485071,0.163474,4.527890,0.850460,1.000000,1,0,0.650000,1,1,0.666667,entailment,"""So your girl writes that your little farewel...",Your farewell activity didn't go well.
1,2,0.200000,0.000000,1.000000,0.615457,0.149428,4.527443,0.879779,1.000000,1,0,0.590909,0,0,0.800000,entailment,"Under Deng Xiaoping, Beijing actively sought t...",Beijing sought to create a good relationship.
2,2,0.250000,0.000000,0.500000,0.535303,0.038836,4.411816,0.731585,0.888889,2,0,0.497076,0,0,0.750000,entailment,GAO also issued over 160 reports detailing spe...,Improving government readiness was aided by th...
3,1,0.000000,0.000000,1.000000,1.000000,0.340767,4.654967,1.000000,1.000000,0,0,1.000000,0,0,1.000000,entailment,The chain wielder smiled at her.,The chain wielder was smiling.
4,2,0.600000,0.000000,0.500000,0.353553,0.075418,4.190316,0.550815,0.800000,2,0,0.563889,0,0,0.400000,entailment,The celebrity-obsessed magazine surpasses itse...,The magazine publishes a lot of stories on cel...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,2,1.000000,0.000000,0.000000,0.198030,0.011191,3.987838,0.441345,0.600000,2,0,0.408497,0,1,0.000000,contradiction,There are also a couple of small aircraft lyin...,Abandoned aircraft have never been found in th...
596,2,1.000000,0.000000,0.000000,0.182574,0.090118,3.795031,0.510137,1.000000,1,0,0.000000,0,0,0.000000,contradiction,"""The summons was only for Dave Hanson,"" Ser P...",All of them had been invited.
597,-1,0.333333,0.333333,0.333333,0.306186,0.147137,4.739438,1.000000,1.000000,0,0,0.000000,0,0,0.333333,contradiction,The analysis also addresses the various altern...,There are no standards for design.
598,1,0.000000,0.000000,0.666667,0.730297,0.047815,4.365051,0.698024,0.600000,0,0,0.822222,0,0,1.000000,contradiction,Scotland became little more than an English co...,Scotland was far greater than an English county.


In [6]:
df_t.columns

Index(['relation', 'no_matcheadas', 'contradiction', 'jaccard', 'simBoW',
       'sumas', 'mutinf_t', 'max_info_t', 'entail', 'list_comp', 'list_incomp',
       'Jaro-Winkler_rit', 'negT', 'negH', 'overlap_ent', 'clases',
       'sentence_A', 'sentence_B'],
      dtype='object')

# RIT

In [7]:
lista_respuestasOllama=[]
faltantes=[]
for i in range(1):
    for index,strings in df_t[i*600:(i+1)*600].iterrows():
        #print(strings["sentence1"],strings["sentence2"],strings['gold_label'])
        prompt = ''''Analyze the following sentences: 
        Text: '''+strings["sentence_A"]+'''
        Hypothesis: '''+ strings["sentence_B"]+'''
         only responds using the template. '''
        #print(prompt)
        data = {
            "prompt": prompt,
            "model": model1,
            "format": "json",
            "stream": False,
            #"options": {"temperature": 0.2},
        }
        try:
            response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=25)        
            json_data = json.loads(response.text)
            lista_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
            print(index)
        except:
            print("Saltó",index)
            faltantes.append(index)
            lista_respuestasOllama.append("NA")
    with open("resultados/rit_MultiNLI_DEV_600_"+str(i+1)+".pickle", "wb") as f:
        pickle.dump(lista_respuestasOllama, f)
    print("Primer rango: ",(i)*600,(i+1)*600)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# DEVMM de MultiNLI

In [8]:
df_t=pd.read_pickle("../predictions/RL_MultiNLI_DEVMM_600.pickle")

In [9]:
df_t

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
0,2,0.250000,0.0,0.500000,0.500000,0.172598,3.440214,0.750000,1.000000,0,0,0.464286,0,0,0.750000,entailment,"If you help me, I said, to move my livestock t...",Can you help me move my livestock to Chapeltique?
1,2,0.250000,0.0,0.750000,0.564076,0.172460,4.467687,0.875487,1.000000,2,0,0.674272,1,1,0.750000,entailment,Make-believe roles are not equally pleasurable...,Not all make-believe role-playing is equally a...
2,2,0.833333,0.0,0.400000,0.316228,0.009732,3.962749,0.371995,0.500000,2,0,0.458333,0,0,0.166667,entailment,He had an appetite and aptitude for business.,"In business, he found that he could excel; on ..."
3,2,0.250000,0.0,0.333333,0.368721,0.073688,4.783562,1.000000,1.000000,0,0,0.308547,1,0,0.750000,entailment,I see the rather boring point about using alum...,The term prima donna can be used for men.
4,2,0.500000,0.0,0.500000,0.912871,0.258084,4.631388,0.838558,1.000000,1,0,0.944444,0,0,0.500000,entailment,How was it going to high school in Charlotte?,What was going to high school in Charlotte like?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,2,0.222222,0.0,0.500000,0.734847,0.105542,4.605025,0.903593,0.888889,0,0,0.827066,0,0,0.777778,contradiction,The line to the White House shelter conference...,The line to the White House shelter conference...
596,2,0.250000,0.0,0.250000,0.544331,0.026523,4.411366,0.862340,0.833333,1,0,0.703704,0,0,0.750000,contradiction,"Making a marker is a complicated task, even wi...",Making markers is easy even without the help o...
597,2,0.750000,0.0,0.000000,0.327327,0.042057,4.256955,0.560480,0.600000,1,0,0.492063,0,1,0.250000,contradiction,We can and should be proud that the IU School ...,The UI school of dentistry doesn't help anyone.
598,2,1.000000,0.0,0.500000,0.547723,0.082198,3.422672,0.478159,0.400000,0,0,0.588889,0,0,0.000000,contradiction,Representatives from 43 different countries we...,Only 10 different representatives showed up.


In [10]:
df_t.columns

Index(['relation', 'no_matcheadas', 'contradiction', 'jaccard', 'simBoW',
       'sumas', 'mutinf_t', 'max_info_t', 'entail', 'list_comp', 'list_incomp',
       'Jaro-Winkler_rit', 'negT', 'negH', 'overlap_ent', 'clases',
       'sentence_A', 'sentence_B'],
      dtype='object')

# RIT pi fs 1

In [11]:
lista_respuestasOllama=[]
faltantes=[]
for i in range(1):
    for index,strings in df_t[i*600:(i+1)*600].iterrows():
        #print(strings["sentence1"],strings["sentence2"],strings['gold_label'])
        prompt = ''''Analyze the following sentences: 
        Text: '''+strings["sentence_A"]+'''
        Hypothesis: '''+ strings["sentence_B"]+'''
        only responds using the template. '''
        #print(prompt)
        data = {
            "prompt": prompt,
            "model": model1,
            "format": "json",
            "stream": False,
            #"options": {"temperature": 0.2},
        }
        try:
            response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=25)        
            json_data = json.loads(response.text)
            lista_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
            print(index)
        except:
            print("Saltó",index)
            faltantes.append(index)
            lista_respuestasOllama.append("NA")
    with open("resultados/rit_MultiNLI_DEVMM_600_"+str(i+1)+".pickle", "wb") as f:
        pickle.dump(lista_respuestasOllama, f)
    print("Primer rango: ",(i)*600,(i+1)*600)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [12]:
df_t=pd.read_pickle("../predictions/RL_Scitail_DEV_600.pickle")

In [13]:
df_t

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
0,2,0.400000,0.0,0.600000,0.594089,0.140100,4.340529,0.762486,0.666667,0,0,0.400000,0,0,0.600000,entails,Below the earth's crust is a layer known as th...,The mantle layer of earth is located just belo...
1,2,0.375000,0.0,0.166667,0.605406,0.171515,4.381443,0.918428,0.875000,1,0,0.708586,0,0,0.625000,entails,Meiosis is the two-stage process of cell divis...,"Gametogenesis, the production of sperm and egg..."
2,2,0.750000,0.0,0.333333,0.571548,0.151636,4.435317,0.862003,0.857143,1,0,0.586111,0,0,0.250000,entails,The central nervous system acts like a telepho...,The nervous system sends electrical signals to...
3,2,0.333333,0.0,0.666667,0.447214,0.152032,4.516060,0.882395,0.800000,0,0,0.429167,0,0,0.666667,entails,Chemists often use as a unit of pressure the t...,A torr is a unit used to measure pressure.
4,2,0.400000,0.0,0.750000,0.707107,0.055214,4.403386,0.889384,0.800000,0,0,0.775000,0,0,0.600000,entails,The liver makes urea and the kidney puts the u...,Urea is made in the liver and excreted in urine.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,2,0.400000,0.0,0.250000,0.617213,0.178026,4.306672,0.622818,0.600000,0,0,0.456349,0,0,0.600000,neutral,Flagella and cilia are extensions of the plasm...,The plasma membrane controls what goes in and ...
596,2,0.625000,0.0,0.333333,0.354787,0.010448,4.223442,0.669017,0.727273,3,0,0.607912,0,0,0.375000,neutral,Introduction People have wondered for a long t...,This process usually takes the longest amount ...
597,2,0.666667,0.0,0.000000,0.471405,0.189161,3.957897,0.353976,0.375000,1,0,0.370370,0,0,0.333333,neutral,Infancy and Childhood,An individual grows quickly and develops new a...
598,2,0.500000,0.0,0.400000,0.670820,0.165602,4.410536,0.775091,0.833333,1,0,0.716667,0,0,0.500000,neutral,Subject matter wherein the light is randomly s...,Translucent matter transmits light but scatter...


In [14]:
df_t.columns

Index(['relation', 'no_matcheadas', 'contradiction', 'jaccard', 'simBoW',
       'sumas', 'mutinf_t', 'max_info_t', 'entail', 'list_comp', 'list_incomp',
       'Jaro-Winkler_rit', 'negT', 'negH', 'overlap_ent', 'clases',
       'sentence_A', 'sentence_B'],
      dtype='object')

# RIT pi fs 1

In [15]:
lista_respuestasOllama=[]
faltantes=[]
for i in range(1):
    for index,strings in df_t[i*600:(i+1)*600].iterrows():
        #print(strings["sentence1"],strings["sentence2"],strings['gold_label'])
        prompt = ''''Analyze the following sentences: 
        Text: '''+strings["sentence_A"]+'''
        Hypothesis: '''+ strings["sentence_B"]+'''
        only responds using the template. '''
        #print(prompt)
        data = {
            "prompt": prompt,
            "model": model1,
            "format": "json",
            "stream": False,
            #"options": {"temperature": 0.2},
        }
        try:
            response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=25)        
            json_data = json.loads(response.text)
            lista_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
            print(index)
        except:
            print("Saltó",index)
            faltantes.append(index)
            lista_respuestasOllama.append("NA")
    with open("resultados/rit_Scitail_DEV_600_"+str(i+1)+".pickle", "wb") as f:
        pickle.dump(lista_respuestasOllama, f)
    print("Primer rango: ",(i)*600,(i+1)*600)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [16]:
df_t=pd.read_pickle("../predictions/RL_Scitail_TEST_600.pickle")

In [17]:
df_t

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
0,2,0.333333,0.0,0.666667,0.755929,0.232549,4.445956,0.770646,1.000000,1,0,0.000000,0,0,0.666667,entails,Introduction An invertebrate is an animal with...,Backbone is another name for the vertebral col...
1,2,0.333333,0.0,0.000000,0.554700,0.168914,4.650058,1.000000,1.000000,0,0,0.493590,0,0,0.666667,entails,"Composed of DNA, genes are the basic unit of i...",Genes are composed of dna.
2,1,0.000000,0.0,0.333333,0.632456,0.223126,4.411628,0.966823,1.000000,1,0,0.683333,0,0,1.000000,entails,"Luckily, hydrogen is the most overwhelming abu...",The element hydrogen is the most abundant in t...
3,2,0.500000,0.0,0.250000,0.408248,0.105719,4.205463,0.704509,0.833333,2,0,0.578704,0,0,0.500000,entails,The equator is relatively hot because sunlight...,The equator on earth receives the most direct ...
4,2,0.500000,0.0,0.000000,0.296174,0.164021,4.317645,0.978230,1.000000,1,0,0.479310,0,0,0.500000,entails,"The team discovered that an immune cell, calle...",Lyme disease is caused by bacteria.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,2,0.714286,0.0,0.600000,0.547723,0.061328,4.124651,0.513348,0.500000,1,0,0.475000,0,0,0.285714,neutral,Increased fragmentation of habitats and urbani...,A fewer predators is most likely to cause the ...
596,2,0.285714,0.0,0.200000,0.424264,0.128466,4.358496,0.740927,0.875000,2,0,0.459259,0,0,0.714286,neutral,When the North Pole is tilted furthest from th...,The number of daylight hours each day would be...
597,2,1.000000,0.0,0.333333,0.567775,0.130620,4.208711,0.641779,0.571429,0,0,0.569872,0,0,0.000000,neutral,He started studying kinetic gas theory and mov...,The behavior of ideal gases is explained by ki...
598,2,0.846154,0.0,0.125000,0.444444,0.098884,3.536910,0.447358,0.714286,6,0,0.482143,0,0,0.153846,neutral,This property is critical to high temperature ...,You call materials able to conduct electricity...


In [18]:
df_t.columns

Index(['relation', 'no_matcheadas', 'contradiction', 'jaccard', 'simBoW',
       'sumas', 'mutinf_t', 'max_info_t', 'entail', 'list_comp', 'list_incomp',
       'Jaro-Winkler_rit', 'negT', 'negH', 'overlap_ent', 'clases',
       'sentence_A', 'sentence_B'],
      dtype='object')

# RIT pi fs 1

In [ ]:
lista_respuestasOllama=[]
faltantes=[]
for i in range(1):
    for index,strings in df_t[i*600:(i+1)*600].iterrows():
        #print(strings["sentence1"],strings["sentence2"],strings['gold_label'])
        prompt = ''''Analyze the following sentences: 
        Text: '''+strings["sentence_A"]+'''
        Hypothesis: '''+ strings["sentence_B"]+'''
        only responds using the template. '''
        #print(prompt)
        data = {
            "prompt": prompt,
            "model": model1,
            "format": "json",
            "stream": False,
            #"options": {"temperature": 0.2},
        }
        try:
            response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=25)        
            json_data = json.loads(response.text)
            lista_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
            print(index)
        except:
            print("Saltó",index)
            faltantes.append(index)
            lista_respuestasOllama.append("NA")
    with open("resultados/rit_Scitail_TEST_600_"+str(i+1)+".pickle", "wb") as f:
        pickle.dump(lista_respuestasOllama, f)
    print("Primer rango: ",(i)*600,(i+1)*600)

0
1
2
3
4
5
6
7
8
9
Saltó 10
11
12
13


# DEV de SICK

In [40]:
df =pd.read_pickle("../predictions/RL_SICK_DEV_with_prediction.pickle")
df

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,predictions
0,-1,0.200000,0.4,0.333333,0.845154,0.233221,4.727097,1.000000,1.000000,0,0,0.904762,0,0,0.400000,CONTRADICTION,0
1,2,0.800000,0.0,0.250000,0.200000,0.057993,3.973408,0.531105,0.666667,3,0,0.466667,0,0,0.200000,NEUTRAL,2
2,2,0.500000,0.0,0.500000,0.288675,0.117675,4.273848,0.669376,0.666667,1,0,0.527778,0,0,0.500000,NEUTRAL,2
3,2,1.000000,0.0,0.000000,0.000000,0.136313,3.621765,0.295252,0.400000,2,0,0.361111,0,0,0.000000,NEUTRAL,2
4,1,0.000000,0.0,1.000000,1.000000,0.241997,4.697862,1.000000,1.000000,0,0,1.000000,0,0,1.000000,NEUTRAL,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,2,1.000000,0.0,0.000000,0.000000,0.067240,3.754545,0.198049,0.333333,1,0,0.361111,0,0,0.000000,NEUTRAL,2
491,2,0.333333,0.0,0.666667,0.577350,0.275896,4.461859,0.732893,1.000000,1,0,0.638889,0,0,0.666667,NEUTRAL,1
492,2,1.000000,0.0,0.000000,0.000000,0.000000,3.783412,0.098616,0.000000,0,0,0.000000,0,0,0.000000,NEUTRAL,2
493,2,1.000000,0.0,0.000000,0.000000,0.028540,3.768143,0.075811,0.500000,2,0,0.333333,0,0,0.000000,NEUTRAL,2


In [41]:
original =pd.read_csv("../../RIT_ollama/SICK/SICK_DEV.csv")
original

,sentence_A,sentence_B,entailment_label
0,The young boys are playing outdoors and the ma...,There is no boy playing outdoors and there is ...,CONTRADICTION
1,A person in a black jacket is doing tricks on ...,A skilled person is riding a bicycle on one wheel,NEUTRAL
2,Four children are doing backbends in the gym,Four girls are doing backbends and playing out...,NEUTRAL
3,A player is throwing the ball,Two teams are competing in a football match,NEUTRAL
4,Five children are standing in front of a woode...,Five children are standing in a wooden hut,NEUTRAL
...,...,...,...
490,A biker is riding away from a fence,A man is dancing on the road,NEUTRAL
491,A woman is playing an electric guitar,A kid is playing guitar,NEUTRAL
492,The animal is grazing on the grass,The cop is sitting on a police bike,NEUTRAL
493,The dog is snapping at some droplets of water,A girl in a band is playing an instrument,NEUTRAL


In [42]:
df["sentence_A"]=original["sentence_A"]
df["sentence_B"]=original["sentence_B"]

In [43]:
df.columns

Index(['relation', 'no_matcheadas', 'contradiction', 'jaccard', 'simBoW',
       'sumas', 'mutinf_t', 'max_info_t', 'entail', 'list_comp', 'list_incomp',
       'Jaro-Winkler_rit', 'negT', 'negH', 'overlap_ent', 'clases',
       'predictions', 'sentence_A', 'sentence_B'],
      dtype='object')

# RIT pi fs 1

In [44]:
lista_respuestasOllama=[]
faltantes=[]
for i in range(1):
    for index,strings in df[i*600:(i+1)*600].iterrows():
        #print(strings["sentence1"],strings["sentence2"],strings['gold_label'])
        prompt = ''''Analyze the following sentences: 
        Text: '''+strings["sentence_A"]+'''
        Hypothesis: '''+ strings["sentence_B"]+'''
        only responds using the template. '''
        #print(prompt)
        data = {
            "prompt": prompt,
            "model": model1,
            "format": "json",
            "stream": False,
            #"options": {"temperature": 0.2},
        }
        try:
            response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=25)        
            json_data = json.loads(response.text)
            lista_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
            print(index)
        except:
            print("Saltó",index)
            faltantes.append(index)
            lista_respuestasOllama.append("NA")
    with open("resultados/rit_SICK_DEV_"+str(i+1)+".pickle", "wb") as f:
        pickle.dump(lista_respuestasOllama, f)
    print("Primer rango: ",(i)*600,(i+1)*600)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# SICK TEST

In [49]:
df_t=pd.read_pickle("../predictions/RL_SICK_TEST_600.pickle")

# RIT pi fs 1

In [50]:
lista_respuestasOllama=[]
faltantes=[]
for i in range(1):
    for index,strings in df_t[i*600:(i+1)*600].iterrows():
        #print(strings["sentence1"],strings["sentence2"],strings['gold_label'])
        prompt = ''''Analyze the following sentences: 
        Text: '''+strings["sentence_A"]+'''
        Hypothesis: '''+ strings["sentence_B"]+'''
        only responds using the template. '''
        #print(prompt)
        data = {
            "prompt": prompt,
            "model": model1,
            "format": "json",
            "stream": False,
            #"options": {"temperature": 0.2},
        }
        try:
            response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=25)        
            json_data = json.loads(response.text)
            lista_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
            print(index)
        except:
            print("Saltó",index)
            faltantes.append(index)
            lista_respuestasOllama.append("NA")
    with open("resultados/rit_SICK_TEST_600_"+str(i+1)+".pickle", "wb") as f:
        pickle.dump(lista_respuestasOllama, f)
    print("Primer rango: ",(i)*600,(i+1)*600)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# SNLI

In [ ]:
df_t=pd.read_pickle("../predictions/RL_SNLI_DEV_600.pickle")

In [ ]:
df_t

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
0,1,0.000000,0.0,0.500000,0.353553,0.254791,4.161840,0.595915,1.000000,1,0,0.416667,0,0,1.000000,entailment,The dog is walking in the snow.,The snow is outside
1,2,0.250000,0.0,0.666667,0.670820,0.237988,4.470784,0.922773,1.000000,1,0,0.783333,0,0,0.750000,entailment,Two men walking their canoe through the woods.,a couple of men walk their canoe
2,2,0.333333,0.0,0.333333,0.387298,0.073237,4.243051,0.735841,0.500000,0,0,0.522222,0,0,0.666667,entailment,A man in his 30's is sitting on a curb of a bu...,A costumed figure is sitting on the curb.
3,2,0.500000,0.0,0.500000,0.174078,0.109343,4.121398,0.626901,0.666667,1,0,0.314815,0,0,0.500000,entailment,"There is a white building, with several people...",There are four vehicles present.
4,2,0.500000,0.0,0.500000,0.436436,0.184308,4.318469,0.856524,1.000000,1,0,0.325397,0,0,0.500000,entailment,Boy reading book with company of a rolled over...,The kid is with his dog.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,2,0.500000,0.0,0.250000,0.547723,0.084415,4.210647,0.571965,0.400000,0,0,0.588889,0,0,0.500000,contradiction,Two children playing on the floor with toy tra...,The girls slept on the floor next to the toys.
596,2,1.000000,0.0,0.333333,0.333333,0.036740,3.897424,0.336676,0.400000,1,0,0.555556,0,0,0.000000,contradiction,The brown and white dog is playing in the snow.,A dog chasing a squirrel in a dry desert.
597,2,0.750000,0.0,0.500000,0.500000,0.094706,4.179998,0.619938,0.800000,2,0,0.788889,0,0,0.250000,contradiction,A girl in a flower dress is running on sand.,The girl is wearing a green striped dress.
598,2,0.750000,0.0,0.333333,0.288675,0.147231,4.354230,0.669921,0.750000,1,0,0.638889,0,0,0.250000,contradiction,Three men converse in a crowd.,Three men talk while laying down


In [ ]:
df_t.columns

Index(['relation', 'no_matcheadas', 'contradiction', 'jaccard', 'simBoW',
       'sumas', 'mutinf_t', 'max_info_t', 'entail', 'list_comp', 'list_incomp',
       'Jaro-Winkler_rit', 'negT', 'negH', 'overlap_ent', 'clases',
       'sentence_A', 'sentence_B'],
      dtype='object')

# RIT pi fs 1

In [ ]:
lista_respuestasOllama=[]
faltantes=[]
for i in range(1):
    for index,strings in df_t[i*600:(i+1)*600].iterrows():
        #print(strings["sentence1"],strings["sentence2"],strings['gold_label'])
        prompt = ''''Analyze the following sentences: 
        Text: '''+strings["sentence_A"]+'''
        Hypothesis: '''+ strings["sentence_B"]+'''
        only responds using the template. '''
        #print(prompt)
        data = {
            "prompt": prompt,
            "model": model1,
            "format": "json",
            "stream": False,
            #"options": {"temperature": 0.2},
        }
        try:
            response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=25)        
            json_data = json.loads(response.text)
            lista_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
            print(index)
        except:
            print("Saltó",index)
            faltantes.append(index)
            lista_respuestasOllama.append("NA")
    with open("resultados/rit_SNLI_DEV_600_"+str(i+1)+".pickle", "wb") as f:
        pickle.dump(lista_respuestasOllama, f)
    print("Primer rango: ",(i)*600,(i+1)*600)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# TEST de SNLI

In [ ]:
df_t=pd.read_pickle("../predictions/RL_SNLI_TEST_600.pickle")

In [ ]:
df_t

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
0,1,0.000000,0.0,1.000000,0.666667,0.182158,4.758932,1.000000,1.000000,0,0,0.773148,0,0,1.000000,entailment,A small child with a backpack walks down a nar...,A small child is walking on a road
1,2,0.333333,0.0,1.000000,0.589256,0.140694,4.613125,1.000000,1.000000,0,0,0.464286,0,0,0.666667,entailment,A brown and white dog walking away from a blac...,A dog stands in the water.
2,2,0.666667,0.0,0.333333,0.288675,0.108599,4.038265,0.681953,0.666667,1,0,0.555556,0,0,0.333333,entailment,A man in the bucket of a chery-picker cuts dow...,A man is trimming trees.
3,2,0.666667,0.0,0.500000,0.377964,-0.049452,4.145515,0.535615,0.666667,1,0,0.464286,0,0,0.333333,entailment,Three people are making transactions at an aut...,A line of people at an ATM.
4,2,0.666667,0.0,0.000000,0.154303,0.108264,3.879840,0.510767,1.000000,2,0,0.436508,0,0,0.333333,entailment,A woman looks at a picture and framing tools w...,There is a person in a room.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,2,0.400000,0.0,0.750000,0.894427,0.231952,4.379169,0.776339,0.600000,0,0,0.933333,0,0,0.600000,contradiction,There are two people walking in the sand.,There are five people walking in the sand with...
596,2,0.333333,0.0,0.400000,0.524142,0.164122,4.365030,0.806454,1.000000,2,0,0.543040,0,0,0.666667,contradiction,A man with a gray beard and a little boy are s...,The man and the boy are taking a nap on the bu...
597,2,0.750000,0.0,0.333333,0.404520,0.105212,4.266216,0.610287,0.750000,1,0,0.263636,0,0,0.250000,contradiction,"A race car, numbered 104 is rounding a bend as...",People are watching a ball game.
598,2,1.000000,0.0,0.000000,0.154303,-0.003144,3.861699,0.143365,0.250000,1,0,0.311111,0,0,0.000000,contradiction,A woman in a white dress with a white umbrella...,Two sisters are flipping each other off.


In [ ]:
df_t.columns

Index(['relation', 'no_matcheadas', 'contradiction', 'jaccard', 'simBoW',
       'sumas', 'mutinf_t', 'max_info_t', 'entail', 'list_comp', 'list_incomp',
       'Jaro-Winkler_rit', 'negT', 'negH', 'overlap_ent', 'clases',
       'sentence_A', 'sentence_B'],
      dtype='object')

# RIT pi fs 1

In [ ]:
lista_respuestasOllama=[]
faltantes=[]
for i in range(1):
    for index,strings in df_t[i*600:(i+1)*600].iterrows():
        #print(strings["sentence1"],strings["sentence2"],strings['gold_label'])
        prompt = ''''Analyze the following sentences: 
        Text: '''+strings["sentence_A"]+'''
        Hypothesis: '''+ strings["sentence_B"]+'''
        only responds using the template. '''
        #print(prompt)
        data = {
            "prompt": prompt,
            "model": model1,
            "format": "json",
            "stream": False,
            #"options": {"temperature": 0.2},
        }
        try:
            response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=25)        
            json_data = json.loads(response.text)
            lista_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
            print(index)
        except:
            print("Saltó",index)
            faltantes.append(index)
            lista_respuestasOllama.append("NA")
    with open("resultados/rit_SNLI_TEST_600_"+str(i+1)+".pickle", "wb") as f:
        pickle.dump(lista_respuestasOllama, f)
    print("Primer rango: ",(i)*600,(i+1)*600)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27